# Apple FP&A Case Study: Trend, Variance, and Scenario Analysis (2010–2020)

## Goal
The objective of this project is to analyze **10 years of Apple’s quarterly financial statements** and replicate the core responsibilities of a **Financial Analyst**.  

This project demonstrates how to:
- Track **revenue, profitability, and expense trends** over time.  
- Compute key **financial KPIs** such as margins, growth rates, and expense ratios.  
- Perform **budget simulation and variance analysis** to evaluate performance against expectations.  
- Apply **forecasting techniques** to project future results.  
- Run **scenario models** (Base, Best, Worst cases) to assess sensitivity to growth and expenses.  
- Communicate findings through **visualizations and concise commentary**.   

### Step 1 — Setup and Initial Load

Import required libraries, configure display settings, and load the raw Excel file.  
This step ensures we can inspect the structure of the dataset, check available sheets, and preview the first rows before cleaning.


In [10]:
import pandas as pd
import numpy as np

pd.set_option("display.float_format", lambda x: f"{x:,.2f}")

# Path to Excel file
file_path = "../data/apple_income_statements.xlsx" 

# Inspect available sheet names
xls = pd.ExcelFile(file_path)
print("Available sheets:", xls.sheet_names)

# Load the AAPL sheet (income statement)
df_raw = pd.read_excel(file_path, sheet_name="AAPL")

# Preview first 10 rows and shape
print("Shape:", df_raw.shape)
df_raw.head(10)

Available sheets: ['AAPL']
Shape: (112, 42)


,Apple Inc (AAPL),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41
0,All numbers except for ratios and EPS are in t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Report Date,2020-03-28 00:00:00,2019-12-28 00:00:00,2019-09-28 00:00:00,2019-06-29 00:00:00,2019-03-30 00:00:00,2018-12-29 00:00:00,2018-09-29 00:00:00,2018-06-30 00:00:00,2018-03-31 00:00:00,...,2012-06-30 00:00:00,2012-03-31 00:00:00,2011-12-31 00:00:00,2011-09-24 00:00:00,2011-06-25 00:00:00,2011-03-26 00:00:00,2010-12-25 00:00:00,2010-09-25 00:00:00,2010-06-26 00:00:00,2010-03-27 00:00:00
2,Income Statement,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Revenues,58313000,91819000,64040000,53809000,58015000,84310000,62900000,53265000,61137000,...,35023000,39186000,46333000,28270000,28571000,24667000,26741000,20343000,15700000,13499000
4,Cost of Goods,35943000,56602000,39727000,33582000,36194000,52279000,38816000,32844000,37715000,...,20029000,20622000,25630000,16890000,16649000,14449000,16443000,12831000,9564000,7874000
5,Gross Profit,22370000,35217000,24313000,20227000,21821000,32031000,24084000,20421000,23422000,...,14994000,18564000,20703000,11380000,11922000,10218000,10298000,7512000,6136000,5625000
6,General/Admin Expense,4952000,5197000,4578000,4426000,4458000,4783000,4216000,4108000,4150000,...,2545000,2339000,2605000,2025000,1915000,1763000,1896000,1571000,1438000,1220000
7,Sales + Marketing Expense,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Research & Development,4565000,4451000,4110000,4257000,3948000,3902000,3750000,3701000,3378000,...,876000,841000,758000,645000,628000,581000,575000,494000,464000,426000
9,Operating Income,12853000,25569000,15625000,11544000,13415000,23346000,16118000,12612000,15894000,...,11573000,15384000,17340000,8710000,9379000,7874000,7827000,5447000,4234000,3979000


### Step 2 — Clean and Structure the Data

The Excel file contains two metadata rows at the top (title and notes), and the **third row** (row index 2) holds the true headers:  
- The first column is `Report Date` (later renamed to `Account`).  
- The remaining columns are quarterly reporting dates in `DD/MM/YYYY` format.  

To clean the dataset:  
- Load with `header=2` so pandas uses the correct header row.  
- Rename the first column to `Account`.  
- Drop the `Income Statement` label row.  
- Set `Account` as the index.  
- Convert all numeric values for analysis.  


In [11]:
# Load using the correct header row (row 2 in the Excel file)
df = pd.read_excel(file_path, sheet_name="AAPL", header=2)

# Rename first column to Account
df = df.rename(columns={df.columns[0]: "Account"})

# Drop the 'Income Statement' row if present
df = df[df["Account"] != "Income Statement"]

# Set Account as index
df = df.set_index("Account")

# Convert all numeric values
df = df.apply(pd.to_numeric, errors="coerce")

# Preview
df.iloc[:10, :5]


,2020-03-28 00:00:00,2019-12-28 00:00:00,2019-09-28 00:00:00,2019-06-29 00:00:00,2019-03-30 00:00:00
Account,,,,,
Revenues,"58,313,000.00","91,819,000.00","64,040,000.00","53,809,000.00","58,015,000.00"
Cost of Goods,"35,943,000.00","56,602,000.00","39,727,000.00","33,582,000.00","36,194,000.00"
Gross Profit,"22,370,000.00","35,217,000.00","24,313,000.00","20,227,000.00","21,821,000.00"
General/Admin Expense,"4,952,000.00","5,197,000.00","4,578,000.00","4,426,000.00","4,458,000.00"
Sales + Marketing Expense,0.00,0.00,0.00,0.00,0.00
Research & Development,"4,565,000.00","4,451,000.00","4,110,000.00","4,257,000.00","3,948,000.00"
Operating Income,"12,853,000.00","25,569,000.00","15,625,000.00","11,544,000.00","13,415,000.00"
Interest Income/Expense,"757,000.00","785,000.00","810,000.00","866,000.00","1,010,000.00"
Other Income/Expense Net,"-10,000.00","89,000.00","206,000.00","43,000.00","30,000.00"


### Step 3 — Reshape to Time Series

For analysis, we need the dataset in a **time series format**:  
- Rows = reporting dates (quarters)  
- Columns = financial accounts  

We’ll transpose the DataFrame and standardize account names to shorter, clean labels.

In [12]:
# Define mapping from raw labels → clean names
ACCOUNT_MAP = {
    "Revenues": "revenue",
    "Cost of Goods": "cogs",
    "Gross Profit": "gross_profit",
    "Operating Income": "operating_income",
    "Net Income": "net_income",
    "Research & Development": "rnd",
    "General/Admin Expense": "admin_expense",
    "Sales + Marketing Expense": "marketing_expense"
}

# Keep only accounts that exist in the current DataFrame
available_accounts = [a for a in ACCOUNT_MAP if a in df.index]

# Subset and rename
df_subset = df.loc[available_accounts].rename(index=ACCOUNT_MAP)

# Transpose: now rows = dates, columns = accounts
df_tidy = df_subset.T.copy()

# Ensure index is datetime
df_tidy.index = pd.to_datetime(df_tidy.index, errors="coerce")
df_tidy = df_tidy.sort_index()

# Drop duplicate columns (e.g., net_income might appear twice)
df_tidy = df_tidy.loc[:, ~df_tidy.columns.duplicated()]

# Preview
print("Columns:", df_tidy.columns.tolist())
print("Date range:", df_tidy.index.min().date(), "→", df_tidy.index.max().date())
print("Number of quarters:", len(df_tidy))
df_tidy.head()

Columns: ['revenue', 'cogs', 'gross_profit', 'operating_income', 'net_income', 'rnd', 'admin_expense', 'marketing_expense']
Date range: 2010-03-27 → 2020-03-28
Number of quarters: 41


Account,revenue,cogs,gross_profit,operating_income,net_income,rnd,admin_expense,marketing_expense
2010-03-27,"13,499,000.00","7,874,000.00","5,625,000.00","3,979,000.00","3,074,000.00","426,000.00","1,220,000.00",0.00
2010-06-26,"15,700,000.00","9,564,000.00","6,136,000.00","4,234,000.00","3,253,000.00","464,000.00","1,438,000.00",0.00
2010-09-25,"20,343,000.00","12,831,000.00","7,512,000.00","5,447,000.00","4,308,000.00","494,000.00","1,571,000.00",0.00
2010-12-25,"26,741,000.00","16,443,000.00","10,298,000.00","7,827,000.00","6,004,000.00","575,000.00","1,896,000.00",0.00
2011-03-26,"24,667,000.00","14,449,000.00","10,218,000.00","7,874,000.00","5,987,000.00","581,000.00","1,763,000.00",0.00


### Step 4 — Compute Core KPIs

From the time-series dataset, calculate the standard finance KPIs:

- Margins: Gross, Operating, Net (as % of revenue)
- Growth: Revenue and Net Income (quarter-over-quarter)
- Expense Ratios: R&D, Admin, Marketing (as % of revenue)

This produces a single table for variance, forecasting, and charts.


In [13]:
# Copy for KPI calculations
df_kpi = df_tidy.copy()

# safety for division
eps = 1e-9

# margins
if {"gross_profit","revenue"}.issubset(df_kpi.columns):
    df_kpi["gross_margin_pct"] = (df_kpi["gross_profit"] / (df_kpi["revenue"] + eps)) * 100
if {"operating_income","revenue"}.issubset(df_kpi.columns):
    df_kpi["operating_margin_pct"] = (df_kpi["operating_income"] / (df_kpi["revenue"] + eps)) * 100
if {"net_income","revenue"}.issubset(df_kpi.columns):
    df_kpi["net_margin_pct"] = (df_kpi["net_income"] / (df_kpi["revenue"] + eps)) * 100

# growth (QoQ)
df_kpi["revenue_growth_pct"] = df_kpi["revenue"].pct_change() * 100 if "revenue" in df_kpi else np.nan
df_kpi["net_income_growth_pct"] = df_kpi["net_income"].pct_change() * 100 if "net_income" in df_kpi else np.nan

# expense ratios
if "rnd" in df_kpi:
    df_kpi["rnd_ratio_pct"] = (df_kpi["rnd"] / (df_kpi["revenue"] + eps)) * 100
if "admin_expense" in df_kpi:
    df_kpi["admin_ratio_pct"] = (df_kpi["admin_expense"] / (df_kpi["revenue"] + eps)) * 100
if "marketing_expense" in df_kpi:
    df_kpi["marketing_ratio_pct"] = (df_kpi["marketing_expense"] / (df_kpi["revenue"] + eps)) * 100

# preview
df_kpi.head()

Account,revenue,cogs,gross_profit,operating_income,net_income,rnd,admin_expense,marketing_expense,gross_margin_pct,operating_margin_pct,net_margin_pct,revenue_growth_pct,net_income_growth_pct,rnd_ratio_pct,admin_ratio_pct,marketing_ratio_pct
2010-03-27,"13,499,000.00","7,874,000.00","5,625,000.00","3,979,000.00","3,074,000.00","426,000.00","1,220,000.00",0.00,41.67,29.48,22.77,NaN,NaN,3.16,9.04,0.00
2010-06-26,"15,700,000.00","9,564,000.00","6,136,000.00","4,234,000.00","3,253,000.00","464,000.00","1,438,000.00",0.00,39.08,26.97,20.72,16.30,5.82,2.96,9.16,0.00
2010-09-25,"20,343,000.00","12,831,000.00","7,512,000.00","5,447,000.00","4,308,000.00","494,000.00","1,571,000.00",0.00,36.93,26.78,21.18,29.57,32.43,2.43,7.72,0.00
2010-12-25,"26,741,000.00","16,443,000.00","10,298,000.00","7,827,000.00","6,004,000.00","575,000.00","1,896,000.00",0.00,38.51,29.27,22.45,31.45,39.37,2.15,7.09,0.00
2011-03-26,"24,667,000.00","14,449,000.00","10,218,000.00","7,874,000.00","5,987,000.00","581,000.00","1,763,000.00",0.00,41.42,31.92,24.27,-7.76,-0.28,2.36,7.15,0.00


### Step 5 — Export KPIs for Tableau & Prepare Budget Simulation

To enable interactive visualization, we export the KPI dataset to a CSV file.  
This file can be loaded into Tableau to build dashboards for:
- Revenue & Net Income trends
- Margin performance
- Expense ratios
- Growth rates

We also prepare the next step: a simple budget simulation to compare actuals vs expectations.

In [14]:
# Export KPI dataset for Tableau
df_kpi.to_csv("../data/apple_financial_kpis.csv")
print("Exported to apple_financial_kpis.csv")

# --- Budget simulation assumptions ---
rev_growth_assumption = 0.05  # 5% QoQ growth
lookback_years = 3 * 4        # last 3 years of quarters

# Calculate average expense ratios from recent history
avg_rnd_ratio = df_kpi["rnd_ratio_pct"].tail(lookback_years).mean() / 100
avg_admin_ratio = df_kpi["admin_ratio_pct"].tail(lookback_years).mean() / 100
avg_marketing_ratio = df_kpi["marketing_ratio_pct"].tail(lookback_years).mean() / 100
avg_cogs_ratio = (df_kpi["cogs"] / df_kpi["revenue"]).tail(lookback_years).mean()

print("\nAssumptions:")
print(f"Revenue growth per quarter: {rev_growth_assumption*100:.1f}%")
print(f"Avg COGS ratio: {avg_cogs_ratio:.2f}")
print(f"Avg R&D ratio: {avg_rnd_ratio:.2%}, Admin: {avg_admin_ratio:.2%}, Marketing: {avg_marketing_ratio:.2%}")



Exported to apple_financial_kpis.csv

Assumptions:
Revenue growth per quarter: 5.0%
Avg COGS ratio: 0.62
Avg R&D ratio: 6.08%, Admin: 7.04%, Marketing: 0.00%


The dataset has been exported for Tableau dashboards.  
With the budget assumptions (growth rate and expense ratios) calculated, we can simulate expected revenue and expenses.  
This sets up the variance analysis between actuals and budget.  


### Step 6 — Budget Simulation & Variance

We’ll create a simple, defensible budget and compare it to actuals.

Assumptions:
- Revenue grows quarter-over-quarter by a fixed rate (default: 5%).
- COGS, R&D, Admin, and Marketing scale with revenue using trailing averages (last 12 quarters by default).

Outputs:
- A budget table aligned to actuals
- Variance ($ and %) for Revenue and Operating Income


In [15]:
# configurable knobs
rev_growth_assumption = 0.05   # 5% QoQ
trailing_quarters = 12         # trailing window for average ratios

df_actual = df_tidy.copy()

# sanity checks for required columns
required_cols = ["revenue", "cogs", "operating_income"]
missing_cols = [c for c in required_cols if c not in df_actual.columns]
if missing_cols:
    raise ValueError(f"Missing required columns for budget: {missing_cols}")

# compute recent average ratios
eps = 1e-9
cogs_ratio = (df_actual["cogs"] / (df_actual["revenue"] + eps)).tail(trailing_quarters).mean()

rnd_ratio  = (df_actual["rnd"] / (df_actual["revenue"] + eps)).tail(trailing_quarters).mean()  if "rnd" in df_actual else 0.0
admin_ratio = (df_actual["admin_expense"] / (df_actual["revenue"] + eps)).tail(trailing_quarters).mean() if "admin_expense" in df_actual else 0.0
mkt_ratio  = (df_actual["marketing_expense"] / (df_actual["revenue"] + eps)).tail(trailing_quarters).mean() if "marketing_expense" in df_actual else 0.0

# build budget frame
budget = pd.DataFrame(index=df_actual.index)
budget["revenue_budget"] = df_actual["revenue"].shift(1) * (1 + rev_growth_assumption)

budget["cogs_budget"]   = budget["revenue_budget"] * cogs_ratio
budget["rnd_budget"]    = budget["revenue_budget"] * rnd_ratio
budget["admin_budget"]  = budget["revenue_budget"] * admin_ratio
budget["mkt_budget"]    = budget["revenue_budget"] * mkt_ratio

budget["operating_income_budget"] = (
    budget["revenue_budget"]
    - budget["cogs_budget"]
    - budget["rnd_budget"]
    - budget["admin_budget"]
    - budget["mkt_budget"]
)

# join actuals and compute variances
df_var = df_actual.join(budget)

def _var_pct(actual, budget):
    return (actual - budget) / (budget + eps) * 100

df_var["revenue_var_pct"]          = _var_pct(df_var["revenue"], df_var["revenue_budget"])
df_var["operating_income_var_pct"] = _var_pct(df_var["operating_income"], df_var["operating_income_budget"])

# helpful tail view (last 8 quarters)
df_var.tail(8)


,revenue,cogs,gross_profit,operating_income,net_income,rnd,admin_expense,marketing_expense,revenue_budget,cogs_budget,rnd_budget,admin_budget,mkt_budget,operating_income_budget,revenue_var_pct,operating_income_var_pct
2018-06-30,"53,265,000.00","32,844,000.00","20,421,000.00","12,612,000.00","11,519,000.00","3,701,000.00","4,108,000.00",0.00,"64,193,850.00","39,712,247.75","3,899,812.55","4,518,371.00",0.00,"16,063,418.70",-17.02,-21.49
2018-09-29,"62,900,000.00","38,816,000.00","24,084,000.00","16,118,000.00","14,125,000.00","3,750,000.00","4,216,000.00",0.00,"55,928,250.00","34,598,898.81","3,397,672.69","3,936,585.56",0.00,"13,995,092.94",12.47,15.17
2018-12-29,"84,310,000.00","52,279,000.00","32,031,000.00","23,346,000.00","19,965,000.00","3,902,000.00","4,783,000.00",0.00,"66,045,000.00","40,857,424.86","4,012,270.95","4,648,666.70",0.00,"16,526,637.50",27.66,41.26
2019-03-30,"58,015,000.00","36,194,000.00","21,821,000.00","13,415,000.00","11,561,000.00","3,948,000.00","4,458,000.00",0.00,"88,525,500.00","54,764,538.79","5,377,973.99","6,230,987.11",0.00,"22,152,000.11",-34.47,-39.44
2019-06-29,"53,809,000.00","33,582,000.00","20,227,000.00","11,544,000.00","10,044,000.00","4,257,000.00","4,426,000.00",0.00,"60,915,750.00","37,684,316.42","3,700,666.12","4,287,637.49",0.00,"15,243,129.96",-11.67,-24.27
2019-09-28,"64,040,000.00","39,727,000.00","24,313,000.00","15,625,000.00","13,686,000.00","4,110,000.00","4,578,000.00",0.00,"56,499,450.00","34,952,260.32","3,432,373.41","3,976,790.24",0.00,"14,138,026.03",13.35,10.52
2019-12-28,"91,819,000.00","56,602,000.00","35,217,000.00","25,569,000.00","22,236,000.00","4,451,000.00","5,197,000.00",0.00,"67,242,000.00","41,597,925.09","4,084,989.38","4,732,919.16",0.00,"16,826,166.38",36.55,51.96
2020-03-28,"58,313,000.00","35,943,000.00","22,370,000.00","12,853,000.00","11,249,000.00","4,565,000.00","4,952,000.00",0.00,"96,409,950.00","59,642,096.87","5,856,958.77","6,785,944.79",0.00,"24,124,949.57",-39.52,-46.72


### Step 7 — Forecasting (CAGR + Linear Trend)

We’ll produce two simple forward projections for Revenue and Net Income:
- CAGR-based forecast from the full history
- Linear regression on time → value

Outputs:
- A forecast table for the next 8 quarters


In [16]:

from sklearn.linear_model import LinearRegression
import numpy as np

forecast_horizon = 8  # quarters

# helper: CAGR
def cagr(series: pd.Series):
    s = series.dropna()
    if len(s) < 2 or s.iloc[0] <= 0:
        return np.nan
    n = len(s) - 1
    return (s.iloc[-1] / s.iloc[0]) ** (1/n) - 1

# helper: linear regression forecast (index as ordinal integers)
def linear_forecast(series: pd.Series, horizon=8):
    y = series.dropna().values
    if len(y) < 3:
        return pd.Series(index=pd.RangeIndex(0), dtype=float)  # not enough history
    X = np.arange(len(y)).reshape(-1,1)
    model = LinearRegression().fit(X, y)
    Xf = np.arange(len(y), len(y)+horizon).reshape(-1,1)
    yf = model.predict(Xf)
    # build future index as quarterly periods after last date
    last_date = series.dropna().index[-1]
    future_index = pd.date_range(last_date, periods=horizon+1, freq="Q")[1:]
    return pd.Series(yf, index=future_index)

# build forecast table
fc = {}

# revenue: CAGR
rev_cagr = cagr(df_actual["revenue"])
if not np.isnan(rev_cagr):
    last_rev = df_actual["revenue"].dropna().iloc[-1]
    cagr_vals = [last_rev * (1 + rev_cagr) ** i for i in range(1, forecast_horizon+1)]
    cagr_idx = pd.date_range(df_actual.index[-1], periods=forecast_horizon+1, freq="Q")[1:]
    fc["revenue_cagr"] = pd.Series(cagr_vals, index=cagr_idx)

# revenue: linear
fc["revenue_linear"] = linear_forecast(df_actual["revenue"], horizon=forecast_horizon)

# net income: try if available
if "net_income" in df_actual:
    ni_cagr = cagr(df_actual["net_income"])
    if not np.isnan(ni_cagr):
        last_ni = df_actual["net_income"].dropna().iloc[-1]
        ni_vals = [last_ni * (1 + ni_cagr) ** i for i in range(1, forecast_horizon+1)]
        ni_idx = pd.date_range(df_actual.index[-1], periods=forecast_horizon+1, freq="Q")[1:]
        fc["net_income_cagr"] = pd.Series(ni_vals, index=ni_idx)
    fc["net_income_linear"] = linear_forecast(df_actual["net_income"], horizon=forecast_horizon)

# combine
forecast_tbl = pd.concat(fc, axis=1)

# preview
forecast_tbl


C:\Users\aksh0\AppData\Local\Temp\ipykernel_17840\2297358528.py:36: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  cagr_idx = pd.date_range(df_actual.index[-1], periods=forecast_horizon+1, freq="Q")[1:]
C:\Users\aksh0\AppData\Local\Temp\ipykernel_17840\2297358528.py:25: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  future_index = pd.date_range(last_date, periods=horizon+1, freq="Q")[1:]
C:\Users\aksh0\AppData\Local\Temp\ipykernel_17840\2297358528.py:48: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  ni_idx = pd.date_range(df_actual.index[-1], periods=forecast_horizon+1, freq="Q")[1:]
C:\Users\aksh0\AppData\Local\Temp\ipykernel_17840\2297358528.py:25: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  future_index = pd.date_range(last_date, periods=horizon+1, freq="Q")[1:]


,revenue_cagr,revenue_linear,net_income_cagr,net_income_linear
2020-06-30,"60,485,598.12","74,762,417.07","11,619,813.78","15,978,163.41"
2020-09-30,"62,739,141.88","75,973,527.53","12,002,851.13","16,215,751.92"
2020-12-31,"65,076,647.10","77,184,637.98","12,398,514.97","16,453,340.42"
2021-03-31,"67,501,241.99","78,395,748.43","12,807,221.54","16,690,928.92"
2021-06-30,"70,016,171.28","79,606,858.89","13,229,400.78","16,928,517.42"
2021-09-30,"72,624,800.61","80,817,969.34","13,665,496.80","17,166,105.92"
2021-12-31,"75,330,621.02","82,029,079.79","14,115,968.36","17,403,694.43"
2022-03-31,"78,137,253.61","83,240,190.24","14,581,289.33","17,641,282.93"


### Step 8 — Scenario Modeling (Base / Best / Worst)

We’ll project the next 8 quarters under three cases:
- Base: revenue growth = recent CAGR; cost ratios = trailing averages
- Best: +200 bps revenue growth; slight COGS/opex ratio improvements
- Worst: −200 bps revenue growth (floored at 0), slight COGS/opex ratio deterioration

Assumptions:
- Net Income approximated as Operating Income × (1 − tax_rate) with a flat tax proxy (default 20%).


In [17]:
# inputs from earlier steps
base_cogs = cogs_ratio
base_rnd  = rnd_ratio
base_admin = admin_ratio
base_mkt  = mkt_ratio

# revenue CAGR fallback: use observed CAGR else use growth assumption
rev_cagr_obs = cagr(df_actual["revenue"])
base_rev_growth = rev_cagr_obs if not np.isnan(rev_cagr_obs) else rev_growth_assumption
best_rev_growth = base_rev_growth + 0.02
worst_rev_growth = max(base_rev_growth - 0.02, 0.0)

# ratio tweaks (bps)
best_cogs = max(base_cogs - 0.01, 0.0)
worst_cogs = base_cogs + 0.01

best_rnd  = max(base_rnd - 0.001, 0.0)
worst_rnd = base_rnd + 0.001

best_admin = max(base_admin - 0.001, 0.0)
worst_admin = base_admin + 0.001

best_mkt  = max(base_mkt - 0.001, 0.0)
worst_mkt = base_mkt + 0.001

tax_rate = 0.20

def project_scenario(last_revenue, start_date, growth, cogs_r, rnd_r, admin_r, mkt_r, horizon=8):
    idx = pd.date_range(start_date, periods=horizon+1, freq="Q")[1:]
    rev = [last_revenue * (1 + growth) ** i for i in range(1, horizon+1)]
    rev = pd.Series(rev, index=idx)
    cogs = rev * cogs_r
    rnd  = rev * rnd_r
    adm  = rev * admin_r
    mkt  = rev * mkt_r
    op_inc = rev - cogs - rnd - adm - mkt
    net_inc = op_inc * (1 - tax_rate)
    out = pd.DataFrame({
        "revenue": rev,
        "cogs": cogs,
        "rnd": rnd,
        "admin": adm,
        "marketing": mkt,
        "operating_income": op_inc,
        "net_income": net_inc
    })
    out["operating_margin_pct"] = (out["operating_income"] / (out["revenue"] + eps)) * 100
    out["net_margin_pct"] = (out["net_income"] / (out["revenue"] + eps)) * 100
    return out

last_rev = df_actual["revenue"].dropna().iloc[-1]
start_date = df_actual.index[-1]

sc_base  = project_scenario(last_rev, start_date, base_rev_growth,  base_cogs,  base_rnd,  base_admin,  base_mkt,  horizon=forecast_horizon)
sc_best  = project_scenario(last_rev, start_date, best_rev_growth,  best_cogs,  best_rnd,  best_admin,  best_mkt,  horizon=forecast_horizon)
sc_worst = project_scenario(last_rev, start_date, worst_rev_growth, worst_cogs, worst_rnd, worst_admin, worst_mkt, horizon=forecast_horizon)

# combine for quick comparison on final quarter
scenario_final = pd.DataFrame({
    "Base_net_income": sc_base["net_income"].iloc[-1],
    "Best_net_income": sc_best["net_income"].iloc[-1],
    "Worst_net_income": sc_worst["net_income"].iloc[-1],
}, index=["Final Quarter"])

scenario_final


C:\Users\aksh0\AppData\Local\Temp\ipykernel_17840\3711179821.py:29: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  idx = pd.date_range(start_date, periods=horizon+1, freq="Q")[1:]
C:\Users\aksh0\AppData\Local\Temp\ipykernel_17840\3711179821.py:29: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  idx = pd.date_range(start_date, periods=horizon+1, freq="Q")[1:]
C:\Users\aksh0\AppData\Local\Temp\ipykernel_17840\3711179821.py:29: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  idx = pd.date_range(start_date, periods=horizon+1, freq="Q")[1:]


,Base_net_income,Best_net_income,Worst_net_income
Final Quarter,"15,642,014.57","19,098,047.70","12,690,470.93"


### Step 9 — Export Outputs

Export all key tables to CSV

In [18]:
df_kpi.to_csv("../data/apple_kpis.csv")
df_var.to_csv("../data/apple_actual_vs_budget.csv")
forecast_tbl.to_csv("../data/apple_forecasts.csv")

sc_base.to_csv("../data/apple_scenario_base.csv")
sc_best.to_csv("../data/apple_scenario_best.csv")
sc_worst.to_csv("../data/apple_scenario_worst.csv")

print("Exported: apple_kpis.csv, apple_actual_vs_budget.csv, apple_forecasts.csv, apple_scenario_*.csv")


Exported: apple_kpis.csv, apple_actual_vs_budget.csv, apple_forecasts.csv, apple_scenario_*.csv
